In [1]:
import nbimport
import metamagics

In [2]:
import Episode5
dir(Episode5)

ERROR:root:Exception in cell.
Traceback (most recent call last):
  File "/home/jon/git/ohop/ohop/nbimport.py", line 73, in load_module
    exec(code, manager.namespace)
  File "<string>", line 1, in <module>
  File "<string>", line 1
    the_possibly_worst_answer = 0
                              ^
SyntaxError: invalid syntax
ERROR:root:Exception in cell.
Traceback (most recent call last):
  File "/home/jon/git/ohop/ohop/nbimport.py", line 73, in load_module
    exec(code, manager.namespace)
  File "<string>", line 2, in <module>
  File "<string>", line 1
    not_a_good_answer = -1
                      ^
SyntaxError: invalid syntax
ERROR:root:Exception in cell.
Traceback (most recent call last):
  File "/home/jon/git/ohop/ohop/nbimport.py", line 73, in load_module
    exec(code, manager.namespace)
  File "<string>", line 1, in <module>
NameError: name '_13' is not defined
ERROR:root:Exception in cell.
Traceback (most recent call last):
  File "/home/jon/git/ohop/ohop/nbimport.py", lin

41
0
The metamagics extension is already loaded. To reload it, use:
  %reload_ext metamagics
The cell is afoot!
Name(id='print', ctx=Load())
Name(id='curious', ctx=Store())
I have a store node: Store()
Name(id='curious', ctx=Load())
Name(id='print', ctx=Load())
Name(id='curious', ctx=Store())
I have a store node: Store()
Silly snake, Python is for the curious!
Go to grad school!


['MetaMagicsPlay',
 'MyVisitor',
 '__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'a_foot',
 'ast',
 'eval_ast',
 'exec_ast',
 'get_ipython',
 'nbimport',
 'ouroboros']

In [4]:
%%mycell Episode5.ouroboros
def foo():
  print('boo')

Name(id='print', ctx=Load())


In [5]:
foo

NameError: name 'foo' is not defined

So, our `metamagics` module doesn't pass the shell to the metamagic function...we've "fixed" that now...

In [6]:
%reload_ext metamagics

In [7]:
def ouroboros2(cell, shell):
    shell.ex(cell)

In [8]:
%%mycell ouroboros2
def oof(zab):
    return zab * 42


In [9]:
oof

<function __main__.oof(zab)>

Let's check to make sure we haven't broken the original `%%mycell` goodness...

In [11]:
%%mycell Episode5.ouroboros
print('It ran...')
oof2 = 99

Name(id='print', ctx=Load())
Name(id='oof2', ctx=Store())
I have a store node: Store()
It ran...


In [12]:
oof2 # Expecting exception...

NameError: name 'oof2' is not defined

In [15]:
import ast
stmt_subclasses = [obj for obj in ast.__dict__.values() if isinstance(obj, type) and issubclass(obj, ast.stmt)]
expr_subclasses = [obj for obj in ast.__dict__.values() if isinstance(obj, type) and issubclass(obj, ast.expr)]

In [16]:
expr_subclasses

[ast.expr,
 ast.BoolOp,
 ast.NamedExpr,
 ast.BinOp,
 ast.UnaryOp,
 ast.Lambda,
 ast.IfExp,
 ast.Dict,
 ast.Set,
 ast.ListComp,
 ast.SetComp,
 ast.DictComp,
 ast.GeneratorExp,
 ast.Await,
 ast.Yield,
 ast.YieldFrom,
 ast.Compare,
 ast.Call,
 ast.FormattedValue,
 ast.JoinedStr,
 ast.Constant,
 ast.Attribute,
 ast.Subscript,
 ast.Starred,
 ast.Name,
 ast.List,
 ast.Tuple,
 ast.Slice,
 ast.Num,
 ast.Str,
 ast.Bytes,
 ast.NameConstant,
 ast.Ellipsis]

So, we've created two lists of various AST classes...let's use them to generate code:

In [17]:
# We start with a loop:
for expr_class in expr_subclasses:
    name = expr_class.__name__
    if name != 'expr':
        print(name)

BoolOp
NamedExpr
BinOp
UnaryOp
Lambda
IfExp
Dict
Set
ListComp
SetComp
DictComp
GeneratorExp
Await
Yield
YieldFrom
Compare
Call
FormattedValue
JoinedStr
Constant
Attribute
Subscript
Starred
Name
List
Tuple
Slice
Num
Str
Bytes
NameConstant
Ellipsis


In [18]:
# Big whoop, right?  Well:
print('class ExprVisitor(ast.NodeVisitor):')
for expr_class in expr_subclasses:
    name = expr_class.__name__
    if name != 'expr':
        print(f'''    def visit_{name}(self, node):
        print(f'Visiting {{node}}')
''')

class ExprVisitor(ast.NodeVisitor):
    def visit_BoolOp(self, node):
        print(f'Visiting {node}')

    def visit_NamedExpr(self, node):
        print(f'Visiting {node}')

    def visit_BinOp(self, node):
        print(f'Visiting {node}')

    def visit_UnaryOp(self, node):
        print(f'Visiting {node}')

    def visit_Lambda(self, node):
        print(f'Visiting {node}')

    def visit_IfExp(self, node):
        print(f'Visiting {node}')

    def visit_Dict(self, node):
        print(f'Visiting {node}')

    def visit_Set(self, node):
        print(f'Visiting {node}')

    def visit_ListComp(self, node):
        print(f'Visiting {node}')

    def visit_SetComp(self, node):
        print(f'Visiting {node}')

    def visit_DictComp(self, node):
        print(f'Visiting {node}')

    def visit_GeneratorExp(self, node):
        print(f'Visiting {node}')

    def visit_Await(self, node):
        print(f'Visiting {node}')

    def visit_Yield(self, node):
        print(f'Visiting {n

In [27]:
# But then we'd have to copy and paste, let's avoid that:
from io import StringIO
generated_code = StringIO()
print('class ExprVisitor(ast.NodeVisitor):', file=generated_code)
for expr_class in expr_subclasses:
    name = expr_class.__name__
    if name != 'expr':
        print(f'''    def visit_{name}(self, node):
        print(f'Visiting {{node}}')
        return self.generic_visit(node)
''', file=generated_code)
generated_src = generated_code.getvalue()
shell = get_ipython()
shell.ex(generated_src)
print(generated_src)

class ExprVisitor(ast.NodeVisitor):
    def visit_BoolOp(self, node):
        print(f'Visiting {node}')
        return self.generic_visit(node)

    def visit_NamedExpr(self, node):
        print(f'Visiting {node}')
        return self.generic_visit(node)

    def visit_BinOp(self, node):
        print(f'Visiting {node}')
        return self.generic_visit(node)

    def visit_UnaryOp(self, node):
        print(f'Visiting {node}')
        return self.generic_visit(node)

    def visit_Lambda(self, node):
        print(f'Visiting {node}')
        return self.generic_visit(node)

    def visit_IfExp(self, node):
        print(f'Visiting {node}')
        return self.generic_visit(node)

    def visit_Dict(self, node):
        print(f'Visiting {node}')
        return self.generic_visit(node)

    def visit_Set(self, node):
        print(f'Visiting {node}')
        return self.generic_visit(node)

    def visit_ListComp(self, node):
        print(f'Visiting {node}')
        return self.gener

In [29]:
# Same as it always was...but...
visitor = ExprVisitor() # I can do this!

In [30]:
def run_visitor(src):
    src_ast = ast.parse(src, mode='exec')
    visitor.visit(src_ast)
run_visitor(generated_src)

Visiting <ast.Attribute object at 0xffff84ee8ee0>
Visiting <ast.Name object at 0xffff84ee8eb0>
Visiting <ast.Call object at 0xffff84ee8d90>
Visiting <ast.Name object at 0xffff84ee8d60>
Visiting <ast.JoinedStr object at 0xffff84ee8d30>
Visiting <ast.Constant object at 0xffff84ee8d00>
Visiting <ast.FormattedValue object at 0xffff84ee8cd0>
Visiting <ast.Name object at 0xffff84ee8ca0>
Visiting <ast.Call object at 0xffff84ee8be0>
Visiting <ast.Attribute object at 0xffff84ee8bb0>
Visiting <ast.Name object at 0xffff84ee8b80>
Visiting <ast.Name object at 0xffff84ee8b50>
Visiting <ast.Call object at 0xffff84ee8a00>
Visiting <ast.Name object at 0xffff84ee89d0>
Visiting <ast.JoinedStr object at 0xffff84ee89a0>
Visiting <ast.Constant object at 0xffff84ee8970>
Visiting <ast.FormattedValue object at 0xffff84ee8940>
Visiting <ast.Name object at 0xffff84ee8910>
Visiting <ast.Call object at 0xffff84ee8850>
Visiting <ast.Attribute object at 0xffff84ee8820>
Visiting <ast.Name object at 0xffff84ee87f0>
Vi